In [80]:
import numpy as np 
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt
delta =0.1
number_of_particles=5000
number_of_pairs = 500
large_or_small=1
time_to_observe=100

In [81]:
class Particle:

    def __init__(self, time, vel, pos):
        self.time = time
        self.vel = vel
        self.pos = pos 
        # vel is a numpy array representing velocity
        # pos is a numpy array representing position
    def display_info (self):
        print('Time: %.2f' % self.time)
        print('Position: %.2f, %.2f' % (self.pos[0], self.pos[1]))
        print('Velocity: %.2f, %.2f' % (self.vel[0], self.vel[1]))        
    def increase_time (self):
        self.time = self.time+ delta

In [82]:
def collide_particle (particle1, particle2):
    import random
    angle_of_attack = 2*np.pi* random.random()
    v1=particle1.vel
    v2=particle2.vel
    pos_diff=np.array([np.cos(angle_of_attack),np.sin(angle_of_attack)])
    value=np.dot(v1-v2,pos_diff)
    new_v1=v1-value*pos_diff
    new_v2=v2+value*pos_diff
    particle1.vel=new_v1
    particle2.vel=new_v2
    particle1.increase_time()
    particle2.increase_time()

In [83]:
class Node:
    
    def __init__ (self, time, pos, particles):
        self.time = time
        self.pos  = pos
        self.particles = particles
        # particles is an numpy array of Particles
    def display_info (self):
        print('Time: %.2f' % self.time)
        print('Position: %.2f, %.2f' % (self.pos[0], self.pos[1]))
        print('The particles within this node: ')
        for i in self.particles:
            print('++++++++++++++++++++++++++++++++++++++++++++')
            i.display_info()
            print('++++++++++++++++++++++++++++++++++++++++++++')        
    def collide_node (self,number_of_pairs):
        # This function takes in the number of pairs of particles to collide
        self.time=self.time+ delta
        index=list(range(len(self.particles)))
        pairs=[]
        # Record the pairs of indexes of particles to collide in the list pairs
        import random
        for i in range(number_of_pairs):
            new_pair=list(random.sample(index,2))
            pairs.append(new_pair)
            index=list(set(index)-set(new_pair))
        for i in pairs:
            collide_particle(self.particles[i[0]],self.particles[i[1]])   
            
    def velocity_space(self):
        space=[] # The velocity space 
        for i in self.particles:
            space.append(i.vel)
        space=np.array(space)
        return space
    
    def hist (self,number,):
        # Plot a 2D histograph for the distribution of velocity at this node
        space=self.velocity_space()
        x=space[:,0]
        y=space[:,1]
        if(large_or_small>0):
            plt.hist2d(x, y,bins=(100,100))
        else:
            plt.hist2d(x, y,bins=(10,10))
        plt.title("2D Histogram #"+str(number))
        plt.savefig("simu_hist"+str(number)+".png")    
    def mean_speed_L1(self):
        space=self.velocity_space()
        speed_sum=0
        length=len(space)
        for i in space:
            speed_sum+=np.sqrt(i[0]**2+i[1]**2)
        return speed_sum/length
    def rel_velocity(self):
        index=list(range(0,len(self.particles) ) )
        comb=list(combinations(index,2))
        number_of_comb=len(comb)
        vel_sum=0
        for i in comb:
            p1=self.particles[i[0]]
            p2=self.particles[i[1]]
            v1=p1.vel
            v2=p2.vel
            diff=np.linalg.norm(np.array(v1)-np.array(v2))
            vel_sum+=diff
        return vel_sum/number_of_comb

    def mean_velocity(self):
        space=self.velocity_space()
        return [np.mean(space[:,0]),np.mean(space[:,1])]

    def kinetic_energy(self):
        kinetic_energy=0
        for i in self.particles:
            kinetic_energy+= 0.5*(i.vel[0]**2+i.vel[1]**2)
        return kinetic_energy
    
    def potential_energy(self):
        m_v=self.mean_velocity()
        number_of_particles=len(self.particles)
        ke=self.kinetic_energy()
        mean_energy = 0.5 * number_of_particles * (m_v[0]**2 + m_v[1]**2 )
        return ke - mean_energy


In [84]:
#Generate a new node named Node1, whose position is [1,1]
Node1=Node(0,[1,1],[])

In [85]:
# We initialize Node1 with 50 particles of the same velocity [1,1]
test_velocities=[]
for i in range(50):
    test_velocities.append([1,1])
test_velocities=np.array(test_velocities)
for i in range(50):
    new_particle= Particle(0,test_velocities[i],np.array([1,1]))
    lst=Node1.particles
    lst.append(new_particle)
    Node1.particles=lst
print(Node1.mean_speed_L1())
print(Node1.rel_velocity())
print(Node1.mean_velocity())
print(Node1.kinetic_energy())
print(Node1.potential_energy())

1.414213562373094
0.0
[1.0, 1.0]
50.0
0.0
